# 05tools_2b: Understanding Model Performance and Fairness with the Language Interpretability Tool (LIT)

The [Language Interpretability Tool (LIT)](https://pair-code.github.io/lit/) helps understand a model behavior across a wide range of inputs.  In this notebook the 05 series models will be evaluated with the LIT tool.  While it might sound like LIT is specific to language models,it actually work well with any model!

This notebook will show how to connect the tool to the model multiple ways and how to use the tool to evaluate the model.


---
## Setup

inputs:

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [2]:
REGION = 'us-central1'
DATANAME = 'fraud'
NOTEBOOK = '05tools_2b'
SERIES = '05'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters

packages:

In [96]:
from google.cloud import aiplatform
from google.cloud import bigquery

from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

import witwidget
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

import numpy as np

clients:

In [97]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client(project=PROJECT_ID)

parameters:

In [98]:
BUCKET = PROJECT_ID
DIR = f"temp/{NOTEBOOK}"

environment:

In [99]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Get Vertex AI Endpoint And Deployed Model

In [100]:
endpoints = aiplatform.Endpoint.list(filter = f"display_name={SERIES}_{DATANAME}")
endpoint = endpoints[0]

In [101]:
endpoint.display_name

'05_fraud'

In [102]:
model = aiplatform.Model(
    model_name = endpoint.list_models()[0].model+f'@{endpoint.list_models()[0].model_version_id}'
)

In [103]:
model.display_name

'05i_fraud'

In [104]:
model.versioned_resource_name

'projects/1026793852137/locations/us-central1/models/model_05i_fraud@1'

In [105]:
model.uri

'gs://statmike-mlops-349915/fraud/models/05i/20220728003419/18/model'

## Get Data for Model Exploration
Retrive the test data for this series:

In [106]:
pred = bq.query(query = f"SELECT * EXCEPT({VAR_TARGET}), {VAR_TARGET} FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' ORDER BY {VAR_TARGET} DESC").to_dataframe()
pred = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+['splits'])]]
pred.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,148074,-2.219219,0.727831,-5.458230,5.924850,3.932464,-3.085984,-1.677870,0.865075,-3.177260,...,0.417472,-0.817343,-0.028752,0.025723,-0.825835,-0.013089,0.413291,-0.131387,0.0,1
1,129668,0.753356,2.284988,-5.164492,3.831112,-0.073622,-1.316596,-1.855495,0.831079,-1.567514,...,0.382007,0.033958,0.187697,0.358433,-0.488934,-0.258802,0.296145,-0.047174,2.0,1
2,56887,-0.075483,1.812355,-2.566981,4.127549,-1.628532,-0.805895,-3.390135,1.019353,-2.451251,...,0.794372,0.270471,-0.143624,0.013566,0.634203,0.213693,0.773625,0.387434,5.0,1
3,146998,-2.064240,2.629739,-0.748406,0.694992,0.418178,1.392520,-1.697801,-6.333065,1.724184,...,6.215514,-1.276909,0.459861,-1.051685,0.209178,-0.319859,0.015434,-0.050117,8.0,1
4,78725,-4.312479,1.886476,-2.338634,-0.475243,-1.185444,-2.112079,-2.122793,0.272565,0.290273,...,0.550541,-0.067870,-1.114692,0.269069,-0.020572,-0.963489,-0.918888,0.001454,60.0,1


In [107]:
len(pred.index)

28522

In [108]:
pred.shape

(28522, 31)

## Python Function For Predictions
The WIT tool connects to models for prediction is several ways.  To demonstrate custom prediction functions, this section builds a Python function that calls a Vertex AI Endpoint.

Try 1: Dictionaries

In [136]:
newobs_dicts = pred.to_dict(orient='records')
newobs_dicts[0]

{'Time': 148074,
 'V1': -2.21921860215056,
 'V2': 0.7278314111063922,
 'V3': -5.45822994652182,
 'V4': 5.92484984705884,
 'V5': 3.9324638237634395,
 'V6': -3.0859842366267003,
 'V7': -1.67786998770016,
 'V8': 0.865074610405235,
 'V9': -3.1772602889458597,
 'V10': -3.4192073840566404,
 'V11': 3.6931739422441203,
 'V12': -3.97843975507806,
 'V13': -1.71859087457346,
 'V14': -8.636297393652589,
 'V15': -0.24296482145526502,
 'V16': 1.17488417316765,
 'V17': 2.13460635695284,
 'V18': 2.59436483300614,
 'V19': -1.25758897993879,
 'V20': 0.9647718037347099,
 'V21': 0.41747174595057,
 'V22': -0.8173433840569749,
 'V23': -0.0287524020141088,
 'V24': 0.0257225108657227,
 'V25': -0.8258353432218559,
 'V26': -0.0130890304987416,
 'V27': 0.413291188715315,
 'V28': -0.131387346404896,
 'Amount': 0.0,
 'Class': 1}

In [110]:
def remote_predictor_dicts(obs):
    if type(obs) is dict: obs = [obs]
    predictions = []
    for ob in obs:
        ob.pop(VAR_TARGET, None)
        instances = [json_format.ParseDict(ob, Value())]
        predictions.append(endpoint.predict(instances=instances).predictions[0])
    return predictions

In [137]:
remote_predictor_dicts(newobs_dicts[0:2])

[[0.00166473, 0.998335302], [0.0168009363, 0.98319906]]